# Neural Machine Translation

Welcome to your first programming assignment for this week! 

You will build a Neural Machine Translation (NMT) model to translate human readable dates ("25th of June, 2009") into machine readable dates ("2009-06-25"). You will do this using an attention model, one of the most sophisticated sequence to sequence models. 

This notebook was produced together with NVIDIA's Deep Learning Institute. 

Let's load all the packages you will need for this assignment.

In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## 1 - Translating human readable dates into machine readable dates

The model you will build here could be used to translate from one language to another, such as translating from English to Hindi. However, language translation requires massive datasets and usually takes days of training on GPUs. To give you a place to experiment with these models even without using massive datasets, we will instead use a simpler "date translation" task. 

The network will input a date written in a variety of possible formats (*e.g. "the 29th of August 1958", "03/30/1968", "24 JUNE 1987"*) and translate them into standardized, machine readable dates (*e.g. "1958-08-29", "1968-03-30", "1987-06-24"*). We will have the network learn to output dates in the common machine-readable format YYYY-MM-DD. 



<!-- 
Take a look at [nmt_utils.py](./nmt_utils.py) to see all the formatting. Count and figure out how the formats work, you will need this knowledge later. !--> 

### 1.1 - Dataset

We will train the model on a dataset of 10000 human readable dates and their equivalent, standardized, machine readable dates. Let's run the following cells to load the dataset and print some examples. 

In [2]:
#m = 10000
#dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

In [3]:
import requests, zipfile, io
#https://github.com/meethariprasad/phd/raw/master/assignments/NLP/Translation/hin.zip
r = requests.get("http://www.manythings.org/anki/hin-eng.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
os.listdir()

['english-hindi-train.pkl',
 'english-german.pkl',
 'enghindi.txt',
 'main_model_weights.h5',
 'main_model.h5',
 'english-hindi.pkl',
 'model.h5',
 'english-german-train.pkl',
 'final_model_weights.h5',
 'images',
 'hin.txt',
 'english-hindi-both.pkl',
 'model.bin',
 'nmt_utils.py',
 'english-german-both.pkl',
 'english-hindi-test.pkl',
 '__pycache__',
 'Neural+machine+translation+with+attention+-+v4.ipynb',
 'english-german-test.pkl',
 '_about.txt']

In [4]:
file=open("hin.txt",'r',encoding='utf-8')
content=file.read()
file.close()

In [5]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_punc = re.compile('[।%s]' % re.escape(string.punctuation))
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# tokenize on white space
			line = line.split()
			# remove punctuation from each token
			line = [re_punc.sub('', w) for w in line]
			# remove tokens with numbers in them
			#line = [word for word in line if word.isalpha()]
			#line=re.sub('[।]', '', line)
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
filename = 'hin.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
print ("Number of clean pairs",clean_pairs.shape[0])
save_clean_data(clean_pairs, 'english-german.pkl')
# spot check
for i in range(10):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Number of clean pairs 2867
Saved: english-german.pkl
[Help] => [बचाओ]
[Jump] => [उछलो]
[Jump] => [कूदो]
[Jump] => [छलांग]
[Hello] => [नमस्ते]
[Hello] => [नमस्कार]
[Cheers] => [वाहवाह]
[Cheers] => [चियर्स]
[Got it] => [समझे कि नहीं]
[Im OK] => [मैं ठीक हूँ]


In [6]:
from pickle import load
from pickle import dump
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-german.pkl')

# reduce dataset size
n_sentences = raw_dataset.shape[0]
print (n_sentences)
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:2800], dataset[2800:]
# save
save_clean_data(dataset, 'english-german-both.pkl')
save_clean_data(train, 'english-german-train.pkl')
save_clean_data(test, 'english-german-test.pkl')

2867
Saved: english-german-both.pkl
Saved: english-german-train.pkl
Saved: english-german-test.pkl


In [7]:
dataset[:10]

array([['I worked on the farm all day', 'मैंने पूरे दिन खेत में काम किया'],
       ['Unless it rains I will go too',
        'अगर बारिश नहीं होती है तो मैं भी जाऊँगा'],
       ['How long has she been sick', 'वह कितने दिनों से बीमार है'],
       ['Do you want to come along', 'तुम साथ आना चाहते हो क्या'],
       ['This medicine will take the pain away',
        'यह दवाई दर्द को दूर कर देगी'],
       ['He is always here between 5 and 6 oclock',
        'वह हमेशा यहाँ पाँच से छः बजे के बीच होता है'],
       ['I dont think I can wait that long', 'मैं उतनी देर रुक नहीं सकता'],
       ['School is over at 330', 'स्कूल साढ़े तीन बजे छूटता है'],
       ['I advise you to stop smoking',
        'मेरे ख़याल से तुम्हे सिगरेट पीना छोड़ देना चहिए'],
       ['We bought a round table', 'हमने गोल मेज़ खरीदी']],
      dtype='<U121')

In [8]:
#Converting it to tuples.
dataset_list=(list(tuple(map(tuple, dataset))))

In [9]:
type(dataset_list[1:2])

list

In [10]:
#Creating small subset for test
#dataset=dataset[0:2]
dataset

array([['I worked on the farm all day', 'मैंने पूरे दिन खेत में काम किया'],
       ['Unless it rains I will go too',
        'अगर बारिश नहीं होती है तो मैं भी जाऊँगा'],
       ['How long has she been sick', 'वह कितने दिनों से बीमार है'],
       ..., 
       ['Did you sleep well', 'तुम अच्छे से सोये क्या'],
       ['Whoever wants to come to my party may come',
        'मेरी पार्टी में जो भी आना चाहता है आ सकता है'],
       ['I am hungry', 'मुझे भूख लगी है']],
      dtype='<U121')

In [11]:
#Converting it to tuples.
pad='<pad>'
unk='<unk>'
dataset_list=(list(tuple(map(tuple, dataset))))

In [12]:
english_sentences_list=list(dataset[:,0])
#Add Pad and Unknown.
english_sentences_list.append(pad)
english_sentences_list.append(unk)
english_sentences_list[len(english_sentences_list)-2:len(english_sentences_list)]

['<pad>', '<unk>']

In [13]:
english_unique_words=set((' '.join(english_sentences_list)).split())
english_vocab_len=len(set((' '.join(english_sentences_list)).split()))
english_vocab_len

2620

In [14]:
hindi_sentences_list=list(dataset[:,1])
#Add Pad and Unknown.
hindi_sentences_list.append(pad)
hindi_sentences_list.append(unk)

In [15]:
hindi_unique_words=set((' '.join(hindi_sentences_list)).split())
hindi_vocab_len=len(set((' '.join(hindi_sentences_list)).split()))
hindi_vocab_len

2873

In [16]:
## One thing we can do to improve the model is instead of one hot encodings of words of length vocabulary, get the word2vec vectors for each word with fixed length.
## Another thing that can be done is train only short sentences. We have found that they are resulting in good results.
## There is a suspition that at post BiLSTM it is better to add the input of previously predicted vector. 

In [17]:
# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
			['this', 'is', 'the', 'second', 'sentence'],
			['yet', 'another', 'sentence'],
			['one', 'more', 'sentence'],
			['and', 'the', 'final', 'sentence']]
type(sentences)

def sentences_to_word2vec_input_format(language_sentences_list):
    word2vec_sentence_feed=list()
    for sentence in language_sentences_list:
        word2vec_sentence_feed.append(sentence.split())
    return(word2vec_sentence_feed)
english_sentences_w2v_format=sentences_to_word2vec_input_format(english_sentences_list)
hindi_sentences_w2v_format=sentences_to_word2vec_input_format(hindi_sentences_list)

In [18]:
from gensim.models import Word2Vec
# train model
english_model = Word2Vec(english_sentences_w2v_format, size=2,min_count=1)
english_words_vocab = list(english_model.wv.vocab)
hindi_model = Word2Vec(hindi_sentences_w2v_format, size=2,min_count=1)
english_words_vocab = list(hindi_model.wv.vocab)

In [19]:
# access vector for one word
word="hari"
try:
    print("Found word Shape of word vector",(english_model[word]).shape)
except KeyError:
    print("not found! Assigning Unknown Vector",  (english_model[unk]).shape)

not found! Assigning Unknown Vector (2,)


/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [20]:
#Convert sentences to Xoh format, but with word to vectors rather than one hot code.
#Xoh.shape (2867, 8, 2873)
#Basically we need to take every word of sentence, get a vector of it. 
#Append these vectors rowwise till you reach.
#Append this combined vector to another Global vector List.
Xoh.shape

NameError: name 'Xoh' is not defined

In [ ]:
#from gensim.models.wrappers import FastText

#model = FastText.load_fasttext_format('wiki.simple')

#print(model.most_similar('teacher'))
# Output = [('headteacher', 0.8075869083404541), ('schoolteacher', 0.7955552339553833), ('teachers', 0.733420729637146), ('teaches', 0.6839243173599243), ('meacher', 0.6825737357139587), ('teach', 0.6285147070884705), ('taught', 0.6244685649871826), ('teaching', 0.6199781894683838), ('schoolmaster', 0.6037642955780029), ('lessons', 0.5812176465988159)]


#print(model.similarity('teacher', 'teaches'))
# Output = 0.683924396754

In [ ]:
#Dictionary
english_dictionary=dict(zip(sorted(english_unique_words), list(range(len(english_unique_words)))))
hindi_dictionary=dict(zip(sorted(hindi_unique_words), list(range(len(hindi_unique_words)))))

In [ ]:
#Reverse Dictionary
revere_dictionary_hindi=dict((v,k) for k,v in hindi_dictionary.items())
revere_dictionary_english=dict((v,k) for k,v in english_dictionary.items())

In [ ]:
english_padding_value=english_dictionary['<pad>']
hindi_padding_value=hindi_dictionary['<pad>']

In [ ]:
#This going to be the global variable with maximum number of words found in a sentence
max_english_words=max(len(line.split()) for line in english_sentences_list)
max_hindi_words=max(len(line.split()) for line in hindi_sentences_list)
min_english_words=min(len(line.split()) for line in english_sentences_list)
min_hindi_words=min(len(line.split()) for line in hindi_sentences_list)
average_english_sentence_size=round(0.5*(max_english_words-min_english_words))
average_hindi_sentence_size=round(0.5*(max_hindi_words-min_hindi_words))
print(max_english_words,max_hindi_words,average_english_sentence_size,average_hindi_sentence_size)

In [ ]:
def get_padded_encoding(sentences_list,language_dictionary,max_language_words):
    padding_value=language_dictionary['<pad>']
    language_array=[]
    #Iterate over List.
    for sentence in sentences_list:
        #Replaces English words with English Vocabulary Indexes and Hindi with Hindi Vocabulary Indexes.
        #Remember to have a logic to have if condition in test where, if a word not in dictionary enters, it will be replaced by unk key value.
        single_sentence_array=([language_dictionary[word] for word in sentence.split()])
        #Find the length of english_single_sentence_array
        length_single_sentence=(len(single_sentence_array))
        #So how many times padding dictionary key needs to be appended, if we say maximum length of sentences to be considered is eng_max_len.
        if (max_language_words>length_single_sentence):
            padding_count=(max_language_words-length_single_sentence)
        else:
            padding_count=0
        if (padding_count>0):
            for pad in range(0,padding_count):
                single_sentence_array.append(padding_value)
        else:
            single_sentence_array=single_sentence_array[0:max_language_words]
        #Append to main array
        language_array.append(single_sentence_array)
    #Convert to Numpy array at the end
    language_array=np.array(language_array)
    return(language_array)

In [ ]:
#One of the things that can be done is create different models for different query lengths.

In [ ]:
average_hindi_sentence_size=8
average_english_sentence_size=8
hindi_encoding=get_padded_encoding(hindi_sentences_list,hindi_dictionary,average_hindi_sentence_size)
english_encoding=get_padded_encoding(english_sentences_list,english_dictionary,average_english_sentence_size)
print(hindi_encoding.shape,english_encoding.shape)

In [ ]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
print(a.shape,b.shape,np.stack((a, b)).shape)

In [ ]:
#Verification
#print(english_sentences_list[1],hindi_sentences_list[1])
#print(english_encoding[1],hindi_encoding[1])
#Check if encoding gives back the same answer

def sentences_to_w2vec(language_encoding,revere_dictionary_language,language_model):
    import numpy as np
    sentence_level_w2vec_list=[]
    #arr = np.empty((2,), float)
    number_of_sentences=english_encoding.shape[0]
    for i in range(0,number_of_sentences):
        language_list_padded=[]
        #print (english_encoding[i])
        for key in language_encoding[i]:
            #print(revere_dictionary_english[key])
            word=(revere_dictionary_language[key])
            try:
                #print("Found word Shape of word vector",(english_model[word]).shape,arr.shape)
                language_list_padded.append(language_model[word])
            except KeyError:
                unk='<unk>'
                #print("not found! Assigning Unknown Vector",  (english_model[unk]).shape)
                language_list_padded.append(language_model[unk])
        #print(np.array(language_list_padded))
        sentence_level_w2vec_list.append((np.array(language_list_padded)))
    sentence_level_w2vec=np.array(sentence_level_w2vec_list)
    return(sentence_level_w2vec)
#for sentence in range(0,len(language_list)):
 #   print(language_list[sentence])
#for key in hindi_encoding[1]:
#    print(revere_dictionary_hindi[key])
#english_dictionary['<pad>']
#hindi_dictionary['<pad>']

In [ ]:
print((sentences_to_w2vec(english_encoding,revere_dictionary_english,english_model)).shape)
print((sentences_to_w2vec(hindi_encoding,revere_dictionary_hindi,hindi_model)).shape)

In [ ]:
#We will convert the english and hindi encodings to one hot encodings.
#Please note Input is of the dimension (number of sentences,max_length_language(every column is a word))
#Output is (number of sentences,Max_length_language(every row is a word),length of vocabulary)
#Basically every row of the onehotcode matrix must be for one word.
#How=1 => 1 0 0
#Are=2 => 0 1 0
#You=3 => 0 0 1
#We are trying to translate hindi to english, so our X is Hindi and Y is English
X=hindi_encoding
Y=english_encoding
#Instead of one hot we can use word embeddings.
Xoh=np.array(list(map(lambda x: to_categorical(x, num_classes=len(hindi_dictionary)), X)))
Yoh=np.array(list(map(lambda x: to_categorical(x, num_classes=len(english_dictionary)), Y)))
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

In [ ]:
#Run this if you want word2vec instead of One hot encoding
#Naming it still as X0h and Yoh to avoid changes in too many places further.
Xoh=sentences_to_w2vec(english_encoding,revere_dictionary_english,english_model)
Yoh=sentences_to_w2vec(hindi_encoding,revere_dictionary_hindi,hindi_model)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

In [ ]:
Tx = hindi_encoding.shape[1]
Ty = english_encoding.shape[1]
Tx,Ty

You've loaded:
- `dataset`: a list of tuples of (human readable date, machine readable date)
- `human_vocab`: a python dictionary mapping all characters used in the human readable dates to an integer-valued index 
- `machine_vocab`: a python dictionary mapping all characters used in machine readable dates to an integer-valued index. These indices are not necessarily consistent with `human_vocab`. 
- `inv_machine_vocab`: the inverse dictionary of `machine_vocab`, mapping from indices back to characters. 

Let's preprocess the data and map the raw text data into the index values. We will also use Tx=30 (which we assume is the maximum length of the human readable date; if we get a longer input, we would have to truncate it) and Ty=10 (since "YYYY-MM-DD" is 10 characters long). 

In [ ]:
#Tx = 30
#Ty = 15
#X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)
#X, Y, Xoh, Yoh = preprocess_data(dataset_list, english_dictionary, hindi_dictionary, Tx, Ty)

#print("X.shape:", X.shape)
#print("Y.shape:", Y.shape)
#print("Xoh.shape:", Xoh.shape)
#print("Yoh.shape:", Yoh.shape)

In [ ]:
Xoh[1]

You now have:
- `X`: a processed version of the human readable dates in the training set, where each character is replaced by an index mapped to the character via `human_vocab`. Each date is further padded to $T_x$ values with a special character (< pad >). `X.shape = (m, Tx)`
- `Y`: a processed version of the machine readable dates in the training set, where each character is replaced by the index it is mapped to in `machine_vocab`. You should have `Y.shape = (m, Ty)`. 
- `Xoh`: one-hot version of `X`, the "1" entry's index is mapped to the character thanks to `human_vocab`. `Xoh.shape = (m, Tx, len(human_vocab))`
- `Yoh`: one-hot version of `Y`, the "1" entry's index is mapped to the character thanks to `machine_vocab`. `Yoh.shape = (m, Tx, len(machine_vocab))`. Here, `len(machine_vocab) = 11` since there are 11 characters ('-' as well as 0-9). 


Lets also look at some examples of preprocessed training examples. Feel free to play with `index` in the cell below to navigate the dataset and see how source/target dates are preprocessed. 

In [ ]:
index = 0
#The dataset is english -> Hindi
#Our target is to generate English given Hindi
print("Source:", dataset_list[index][1])
print("Target:", dataset_list[index][0])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

## Tensorflow Implementation

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
import math
import numpy as np
import os
import random
import tensorflow as tf
from matplotlib import pylab
from collections import Counter
import csv

# Seq2Seq Items
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.python.layers.core import Dense

## Input

tf.reset_default_graph()

enc_train_inputs = []
dec_train_inputs = []

# Need to use pre-trained word embeddings
encoder_emb_layer = tf.convert_to_tensor(np.load('de-embeddings.npy'))
decoder_emb_layer = tf.convert_to_tensor(np.load('en-embeddings.npy'))

# Defining unrolled training inputs
for ui in range(source_sequence_length):
    enc_train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size],name='enc_train_inputs_%d'%ui))

dec_train_labels=[]
dec_label_masks = []
for ui in range(target_sequence_length):
    dec_train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size],name='dec_train_inputs_%d'%ui))
    dec_train_labels.append(tf.placeholder(tf.int32, shape=[batch_size],name='dec-train_outputs_%d'%ui))
    dec_label_masks.append(tf.placeholder(tf.float32, shape=[batch_size],name='dec-label_masks_%d'%ui))
    
encoder_emb_inp = [tf.nn.embedding_lookup(encoder_emb_layer, src) for src in enc_train_inputs]
encoder_emb_inp = tf.stack(encoder_emb_inp)

decoder_emb_inp = [tf.nn.embedding_lookup(decoder_emb_layer, src) for src in dec_train_inputs]
decoder_emb_inp = tf.stack(decoder_emb_inp)

enc_train_inp_lengths = tf.placeholder(tf.int32, shape=[batch_size],name='train_input_lengths')
dec_train_inp_lengths = tf.placeholder(tf.int32, shape=[batch_size],name='train_output_lengths')

## Encoder

encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

initial_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)

encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_emb_inp, initial_state=initial_state,
    sequence_length=enc_train_inp_lengths, 
    time_major=True, swap_memory=True)

## Decoder

# Build RNN cell
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

projection_layer = Dense(units=vocab_size, use_bias=True)

# Helper
helper = tf.contrib.seq2seq.TrainingHelper(
    decoder_emb_inp, [tgt_max_sent_length-1 for _ in range(batch_size)], time_major=True)

# Decoder
if decoder_type == 'basic':
    decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_cell, helper, encoder_state,
        output_layer=projection_layer)
    
elif decoder_type == 'attention':
    decoder = tf.contrib.seq2seq.BahdanauAttention(
        decoder_cell, helper, encoder_state,
        output_layer=projection_layer)
    
# Dynamic decoding
outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
    decoder, output_time_major=True,
    swap_memory=True

## Loss & Prediction

logits = outputs.rnn_output

crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=dec_train_labels, logits=logits)
loss = (tf.reduce_sum(crossent*tf.stack(dec_label_masks)) / (batch_size*target_sequence_length))

train_prediction = outputs.sample_id

## Defining Optimizer with Gradient Clipping

print('Defining Optimizer')
# Adam Optimizer. And gradient clipping.
global_step = tf.Variable(0, trainable=False)
inc_gstep = tf.assign(global_step,global_step + 1)
learning_rate = tf.train.exponential_decay(
    0.01, global_step, decay_steps=10, decay_rate=0.9, staircase=True)

with tf.variable_scope('Adam'):
    adam_optimizer = tf.train.AdamOptimizer(learning_rate)

adam_gradients, v = zip(*adam_optimizer.compute_gradients(loss))
adam_gradients, _ = tf.clip_by_global_norm(adam_gradients, 25.0)
adam_optimize = adam_optimizer.apply_gradients(zip(adam_gradients, v))

with tf.variable_scope('SGD'):
    sgd_optimizer = tf.train.GradientDescentOptimizer(learning_rate)

sgd_gradients, v = zip(*sgd_optimizer.compute_gradients(loss))
sgd_gradients, _ = tf.clip_by_global_norm(sgd_gradients, 25.0)
sgd_optimize = sgd_optimizer.apply_gradients(zip(sgd_gradients, v))

sess = tf.InteractiveSession()

## Running the NMT

for step in range(num_steps):

    # input_sizes for each bin: [40]
    # output_sizes for each bin: [60]
    print('.',end='')
    if (step+1)%100==0:
        print('')
        
    sent_ids = np.random.randint(low=0,high=train_inputs.shape[0],size=(batch_size))
    # ====================== ENCODER DATA COLLECTION ================================================
    
    eu_data, eu_labels, _, eu_lengths = enc_data_generator.unroll_batches(sent_ids=sent_ids)
    
    feed_dict = {}
    feed_dict[enc_train_inp_lengths] = eu_lengths
    for ui,(dat,lbl) in enumerate(zip(eu_data,eu_labels)):            
        feed_dict[enc_train_inputs[ui]] = dat                
    
    # ====================== DECODER DATA COLLECITON ===========================
    # First step we change the ids in a batch
    du_data, du_labels, _, du_lengths = dec_data_generator.unroll_batches(sent_ids=sent_ids)
    
    feed_dict[dec_train_inp_lengths] = du_lengths
    for ui,(dat,lbl) in enumerate(zip(du_data,du_labels)):            
        feed_dict[dec_train_inputs[ui]] = dat
        feed_dict[dec_train_labels[ui]] = lbl
        feed_dict[dec_label_masks[ui]] = (np.array([ui for _ in range(batch_size)])<du_lengths).astype(np.int32)
    
# ======================= OPTIMIZATION ==========================
    if step < 10000:
        _,l,tr_pred = sess.run([adam_optimize,loss,train_prediction], feed_dict=feed_dict)
    else:
        _,l,tr_pred = sess.run([sgd_optimize,loss,train_prediction], feed_dict=feed_dict)
        tr_pred = tr_pred.flatten()

## 2 - Neural machine translation with attention

If you had to translate a book's paragraph from French to English, you would not read the whole paragraph, then close the book and translate. Even during the translation process, you would read/re-read and focus on the parts of the French paragraph corresponding to the parts of the English you are writing down. 

The attention mechanism tells a Neural Machine Translation model where it should pay attention to at any step. 


### 2.1 - Attention mechanism

In this part, you will implement the attention mechanism presented in the lecture videos. Here is a figure to remind you how the model works. The diagram on the left shows the attention model. The diagram on the right shows what one "Attention" step does to calculate the attention variables $\alpha^{\langle t, t' \rangle}$, which are used to compute the context variable $context^{\langle t \rangle}$ for each timestep in the output ($t=1, \ldots, T_y$). 

<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **Figure 1**: Neural machine translation with attention</center></caption>



Here are some properties of the model that you may notice: 

- There are two separate LSTMs in this model (see diagram on the left). Because the one at the bottom of the picture is a Bi-directional LSTM and comes *before* the attention mechanism, we will call it *pre-attention* Bi-LSTM. The LSTM at the top of the diagram comes *after* the attention mechanism, so we will call it the *post-attention* LSTM. The pre-attention Bi-LSTM goes through $T_x$ time steps; the post-attention LSTM goes through $T_y$ time steps. 

- The post-attention LSTM passes $s^{\langle t \rangle}, c^{\langle t \rangle}$ from one time step to the next. In the lecture videos, we were using only a basic RNN for the post-activation sequence model, so the state captured by the RNN output activations $s^{\langle t\rangle}$. But since we are using an LSTM here, the LSTM has both the output activation $s^{\langle t\rangle}$ and the hidden cell state $c^{\langle t\rangle}$. However, unlike previous text generation examples (such as Dinosaurus in week 1), in this model the post-activation LSTM at time $t$ does will not take the specific generated $y^{\langle t-1 \rangle}$ as input; it only takes $s^{\langle t\rangle}$ and $c^{\langle t\rangle}$ as input. We have designed the model this way, because (unlike language generation where adjacent characters are highly correlated) there isn't as strong a dependency between the previous character and the next character in a YYYY-MM-DD date. 

- We use $a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}; \overleftarrow{a}^{\langle t \rangle}]$ to represent the concatenation of the activations of both the forward-direction and backward-directions of the pre-attention Bi-LSTM. 

- The diagram on the right uses a `RepeatVector` node to copy $s^{\langle t-1 \rangle}$'s value $T_x$ times, and then `Concatenation` to concatenate $s^{\langle t-1 \rangle}$ and $a^{\langle t \rangle}$ to compute $e^{\langle t, t'}$, which is then passed through a softmax to compute $\alpha^{\langle t, t' \rangle}$. We'll explain how to use `RepeatVector` and `Concatenation` in Keras below. 

Lets implement this model. You will start by implementing two functions: `one_step_attention()` and `model()`.

**1) `one_step_attention()`**: At step $t$, given all the hidden states of the Bi-LSTM ($[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$) and the previous hidden state of the second LSTM ($s^{<t-1>}$), `one_step_attention()` will compute the attention weights ($[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$) and output the context vector (see Figure  1 (right) for details):
$$context^{<t>} = \sum_{t' = 0}^{T_x} \alpha^{<t,t'>}a^{<t'>}\tag{1}$$ 

Note that we are denoting the attention in this notebook $context^{\langle t \rangle}$. In the lecture videos, the context was denoted $c^{\langle t \rangle}$, but here we are calling it $context^{\langle t \rangle}$ to avoid confusion with the (post-attention) LSTM's internal memory cell variable, which is sometimes also denoted $c^{\langle t \rangle}$. 
  
**2) `model()`**: Implements the entire model. It first runs the input through a Bi-LSTM to get back $[a^{<1>},a^{<2>}, ..., a^{<T_x>}]$. Then, it calls `one_step_attention()` $T_y$ times (`for` loop). At each iteration of this loop, it gives the computed context vector $c^{<t>}$ to the second LSTM, and runs the output of the LSTM through a dense layer with softmax activation to generate a prediction $\hat{y}^{<t>}$. 



**Exercise**: Implement `one_step_attention()`. The function `model()` will call the layers in `one_step_attention()` $T_y$ using a for-loop, and it is important that all $T_y$ copies have the same weights. I.e., it should not re-initiaiize the weights every time. In other words, all $T_y$ steps should have shared weights. Here's how you can implement layers with shareable weights in Keras:
1. Define the layer objects (as global variables for examples).
2. Call these objects when propagating the input.

We have defined the layers you need as global variables. Please run the following cells to create them. Please check the Keras documentation to make sure you understand what these layers are: [RepeatVector()](https://keras.io/layers/core/#repeatvector), [Concatenate()](https://keras.io/layers/merge/#concatenate), [Dense()](https://keras.io/layers/core/#dense), [Activation()](https://keras.io/layers/core/#activation), [Dot()](https://keras.io/layers/merge/#dot).

In [ ]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1,activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

Now you can use these layers to implement `one_step_attention()`. In order to propagate a Keras tensor object X through one of these layers, use `layer(X)` (or `layer([X,Y])` if it requires multiple inputs.), e.g. `densor(X)` will propagate X through the `Dense(1)` layer defined above.

In [ ]:
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    print ("s_prev.shape before repeator",s_prev.shape)
    s_prev = repeator(s_prev)
    print ("s_prev.shape after repeator",s_prev.shape)
    print ("a.shape",a.shape)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    print ("concat.shape",concat.shape)
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    print ("e.shape",e.shape)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    print ("alphas.shape",alphas.shape)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    print ("context.shape",context.shape)
    ### END CODE HERE ###
    
    return context

You will be able to check the expected output of `one_step_attention()` after you've coded the `model()` function.

**Exercise**: Implement `model()` as explained in figure 2 and the text above. Again, we have defined global layers that will share weights to be used in `model()`.

In [ ]:
n_a = 500
n_s = 500
post_activation_LSTM_cell = (LSTM(n_s, activation='relu',return_state = True,dropout=0.25))
output_layer = Dense(len(english_dictionary), activation=softmax)

Now you can use these layers $T_y$ times in a `for` loop to generate the outputs, and their parameters will not be reinitialized. You will have to carry out the following steps: 

1. Propagate the input into a [Bidirectional](https://keras.io/layers/wrappers/#bidirectional) [LSTM](https://keras.io/layers/recurrent/#lstm)
2. Iterate for $t = 0, \dots, T_y-1$: 
    1. Call `one_step_attention()` on $[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$ and $s^{<t-1>}$ to get the context vector $context^{<t>}$.
    2. Give $context^{<t>}$ to the post-attention LSTM cell. Remember pass in the previous hidden-state $s^{\langle t-1\rangle}$ and cell-states $c^{\langle t-1\rangle}$ of this LSTM using `initial_state= [previous hidden state, previous cell state]`. Get back the new hidden state $s^{<t>}$ and the new cell state $c^{<t>}$.
    3. Apply a softmax layer to $s^{<t>}$, get the output. 
    4. Save the output by adding it to the list of outputs.

3. Create your Keras model instance, it should have three inputs ("inputs", $s^{<0>}$ and $c^{<0>}$) and output the list of "outputs".

In [ ]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, source_dictionary_size, target_dictionary_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, source_dictionary_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    #out=Input(shape=(1, target_dictionary_size))
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, activation='relu',return_sequences=True,dropout=0.25))(X)
    print("a.shape:",a.shape)
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        print("Before getting Context: a.shape,s.shape",a.shape,s.shape)
        context = one_step_attention(a, s)
        print("context.shape,s.shape,c.shape ",context.shape,s.shape,c.shape)
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s,_, c = post_activation_LSTM_cell(context,initial_state = [s,c])
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        print("Out shape",out.shape)
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X, s0, c0], outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
print (Tx, Ty, n_a, n_s, len(hindi_dictionary), len(english_dictionary))

Run the following cell to create your model.

In [ ]:
model = model(Tx, Ty, n_a, n_s, len(hindi_dictionary), len(english_dictionary))

#You will need copy of the model which will use the weights from model.fit.
#model.load_weights(weightFile)
loaded_model = model

Let's get a summary of the model to check if it matches the expected output.

In [ ]:
model.summary()

**Expected Output**:

Here is the summary you should see
<table>
    <tr>
        <td>
            **Total params:**
        </td>
        <td>
         185,484
        </td>
    </tr>
        <tr>
        <td>
            **Trainable params:**
        </td>
        <td>
         185,484
        </td>
    </tr>
            <tr>
        <td>
            **Non-trainable params:**
        </td>
        <td>
         0
        </td>
    </tr>
                    <tr>
        <td>
            **bidirectional_1's output shape **
        </td>
        <td>
         (None, 30, 128)  
        </td>
    </tr>
    <tr>
        <td>
            **repeat_vector_1's output shape **
        </td>
        <td>
         (None, 30, 128)  
        </td>
    </tr>
                <tr>
        <td>
            **concatenate_1's output shape **
        </td>
        <td>
         (None, 30, 256) 
        </td>
    </tr>
            <tr>
        <td>
            **attention_weights's output shape **
        </td>
        <td>
         (None, 30, 1)  
        </td>
    </tr>
        <tr>
        <td>
            **dot_1's output shape **
        </td>
        <td>
         (None, 1, 128) 
        </td>
    </tr>
           <tr>
        <td>
            **dense_2's output shape **
        </td>
        <td>
         (None, 11) 
        </td>
    </tr>
</table>


As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, a custom [Adam](https://keras.io/optimizers/#adam) [optimizer](https://keras.io/optimizers/#usage-of-optimizers) (`learning rate = 0.005`, $\beta_1 = 0.9$, $\beta_2 = 0.999$, `decay = 0.01`)  and `['accuracy']` metrics:

In [ ]:
from keras.optimizers import RMSprop,Adagrad,Adam,Adamax,Nadam,SGD

In [ ]:
### START CODE HERE ### (≈2 lines)
from keras.optimizers import RMSprop
out = model.compile(optimizer='rmsprop'#(lr=0.001, beta_1=0.7, beta_2=0.8, decay=0.02)
                    ,metrics=['accuracy'],
                    loss='categorical_crossentropy')
out
### END CODE HERE ###

The last step is to define all your inputs and outputs to fit the model:
- You already have X of shape $(m = 10000, T_x = 30)$ containing the training examples.
- You need to create `s0` and `c0` to initialize your `post_activation_LSTM_cell` with 0s.
- Given the `model()` you coded, you need the "outputs" to be a list of 11 elements of shape (m, T_y). So that: `outputs[i][0], ..., outputs[i][Ty]` represent the true labels (characters) corresponding to the $i^{th}$ training example (`X[i]`). More generally, `outputs[i][j]` is the true label of the $j^{th}$ character in the $i^{th}$ training example.

In [ ]:
s0 = np.zeros((len(dataset_list), n_s))
c0 = np.zeros((len(dataset_list), n_s))
outputs = list(Yoh.swapaxes(0,1))

In [ ]:
len(list(Yoh[1])),len(list(Yoh.swapaxes(0,1)))

In [ ]:
Yoh.shape,s0.shape,c0.shape,Yoh.swapaxes(0,1).shape

In [ ]:
#Divide data in to train & test
#How much percentage of total data you need
training_sample_percentage=98
training_sample_count=(round(X.shape[0]*training_sample_percentage/100))
#testing_sample_count=2
#For to cover rest of data
testing_sample_count=X.shape[0]-training_sample_count
testing_sample_index=training_sample_count+testing_sample_count

print("Total Samples,Training,Testing",X.shape[0],training_sample_count,testing_sample_count)
trainXoh=Xoh[0:training_sample_count]
trainYoh=Yoh[0:training_sample_count]
testXoh=Xoh[training_sample_count:testing_sample_index]
testYoh=Yoh[training_sample_count:testing_sample_index]
print("Training X Shape and Y Shape",trainXoh.shape,trainYoh.shape)
print("Testing X Shape and Y Shape",testXoh.shape,testYoh.shape)
train_outputs = list(trainYoh.swapaxes(0,1))
test_outputs = list(testYoh.swapaxes(0,1))

#Run this only to test with couple of samples to check everything is working fine.
#If not needed just change the format to markdown or comment all code below.
#Divide data in to train & test

#Training sample count will alse serve as index
training_sample_count=10
testing_sample_count=2
testing_sample_index=training_sample_count+testing_sample_count

print("Total Samples,Training,Testing",X.shape[0],training_sample_count,testing_sample_count)
trainXoh=Xoh[0:training_sample_count]
trainYoh=Yoh[0:training_sample_count]
testXoh=Xoh[training_sample_count:testing_sample_index]
testYoh=Yoh[training_sample_count:testing_sample_index]
print("Training X Shape and Y Shape",trainXoh.shape,trainYoh.shape)
print("Testing X Shape and Y Shape",testXoh.shape,testYoh.shape)
train_outputs = list(trainYoh.swapaxes(0,1))
test_outputs = list(testYoh.swapaxes(0,1))

In [ ]:
len(dataset_list),training_sample_count,testing_sample_count

In [ ]:
train_s0 = np.zeros((training_sample_count, n_s))
train_c0 = np.zeros((training_sample_count, n_s))
trainX=[trainXoh, train_s0, train_c0]
trainY=train_outputs
test_s0 = np.zeros((testing_sample_count, n_s))
test_c0 = np.zeros((testing_sample_count, n_s))
testX=[testXoh, test_s0, test_c0]
testY=test_outputs
print(s0.shape,c0.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('main_model_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto',save_weights_only=True)
model.fit(trainX, trainY, epochs=2, batch_size=20, validation_data=(testX, testY), callbacks=[checkpoint])
#Hoping to save model without errors.
model.save('main_model.h5')
model.save_weights('final_model_weights.h5')

Let's now fit the model and run it for one epoch.

In [ ]:
#All data model.
#model.fit([Xoh, s0, c0], outputs, epochs=10000, batch_size=20)
#model.save_weights('model_weights.h5')

While training you can see the loss as well as the accuracy on each of the 10 positions of the output. The table below gives you an example of what the accuracies could be if the batch had 2 examples: 

<img src="images/table.png" style="width:700;height:200px;"> <br>
<caption><center>Thus, `dense_2_acc_8: 0.89` means that you are predicting the 7th character of the output correctly 89% of the time in the current batch of data. </center></caption>


We have run this model for longer, and saved the weights. Run the next cell to load our weights. (By training a model for several minutes, you should be able to obtain a model of similar accuracy, but loading our model will save you time.) 

In [ ]:
# model.load_weights('models/model.h5')

In [ ]:
from keras.models import load_model
loaded_model.load_weights('main_model_weights.h5')

You can now see the results on new examples.

In [ ]:
#EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
EXAMPLES=hindi_sentences_list[0:8]
true_test="हमने खरीदी"
EXAMPLES.append(true_test)
EXAMPLES

In [ ]:
EXAMPLES_CODED=get_padded_encoding(EXAMPLES,hindi_dictionary,average_hindi_sentence_size)
print(EXAMPLES_CODED,EXAMPLES_CODED.shape,hindi_encoding.shape)

In [ ]:
i=0
for example in EXAMPLES_CODED:
    iteration=i+1
    source = example
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(hindi_dictionary)), source))).swapaxes(0,1)
    prediction = loaded_model.predict([source,train_s0, train_c0])
    #print ("Prediction, Type & Shape:",prediction,type(prediction),len(prediction))
    prediction = np.argmax(prediction, axis = -1)
    #print ("Prediction, After Argmax:",prediction)
    output = [revere_dictionary_english[int(i)] for i in prediction]
    print("\n ##### \n")
    print("Hindi",EXAMPLES[i])
    if (iteration!=EXAMPLES_CODED.shape[0]):
        print("Expected:",english_sentences_list[i])
    print("Predicted output:", ' '.join(output))
    #print ("Prediction:",list(prediction))
    i=i+1

You can also change these examples to test with your own examples. The next part will give you a better sense on what the attention mechanism is doing--i.e., what part of the input the network is paying attention to when generating a particular output character. 

## 3 - Visualizing Attention (Optional / Ungraded)

Since the problem has a fixed output length of 10, it is also possible to carry out this task using 10 different softmax units to generate the 10 characters of the output. But one advantage of the attention model is that each part of the output (say the month) knows it needs to depend only on a small part of the input (the characters in the input giving the month). We can  visualize what part of the output is looking at what part of the input.

Consider the task of translating "Saturday 9 May 2018" to "2018-05-09". If we visualize the computed $\alpha^{\langle t, t' \rangle}$ we get this: 

<img src="images/date_attention.png" style="width:600;height:300px;"> <br>
<caption><center> **Figure 8**: Full Attention Map</center></caption>

Notice how the output ignores the "Saturday" portion of the input. None of the output timesteps are paying much attention to that portion of the input. We see also that 9 has been translated as 09 and May has been correctly translated into 05, with the output paying attention to the parts of the input it needs to to make the translation. The year mostly requires it to pay attention to the input's "18" in order to generate "2018." 



### 3.1 - Getting the activations from the network

Lets now visualize the attention values in your network. We'll propagate an example through the network, then visualize the values of $\alpha^{\langle t, t' \rangle}$. 

To figure out where the attention values are located, let's start by printing a summary of the model .

In [ ]:
model.summary()

Navigate through the output of `model.summary()` above. You can see that the layer named `attention_weights` outputs the `alphas` of shape (m, 30, 1) before `dot_2` computes the context vector for every time step $t = 0, \ldots, T_y-1$. Lets get the activations from this layer.

The function `attention_map()` pulls out the attention values from your model and plots them.

In [ ]:
test_sentence=str(hindi_sentences_list[1])

In [ ]:
attention_map = plot_attention_map(model, hindi_dictionary, revere_dictionary_english, test_sentence, num = 6, n_s = 128)

On the generated plot you can observe the values of the attention weights for each character of the predicted output. Examine this plot and check that where the network is paying attention makes sense to you.

In the date translation application, you will observe that most of the time attention helps predict the year, and hasn't much impact on predicting the day/month.

### Congratulations!


You have come to the end of this assignment 

<font color='blue'> **Here's what you should remember from this notebook**:

- Machine translation models can be used to map from one sequence to another. They are useful not just for translating human languages (like French->English) but also for tasks like date format translation. 
- An attention mechanism allows a network to focus on the most relevant parts of the input when producing a specific part of the output. 
- A network using an attention mechanism can translate from inputs of length $T_x$ to outputs of length $T_y$, where $T_x$ and $T_y$ can be different. 
- You can visualize attention weights $\alpha^{\langle t,t' \rangle}$ to see what the network is paying attention to while generating each output.

Congratulations on finishing this assignment! You are now able to implement an attention model and use it to learn complex mappings from one sequence to another. 